# <font color ='green' > Common


In [4]:
from collections import Counter , defaultdict

import pandas as pd

pd.set_option('display.max_columns', None)
from pandas import Series as s , DataFrame as df
import numpy as np
import seaborn as sns

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc
%matplotlib inline
rc["figure.figsize"] = 10,6


from scipy import stats as s
from random import randint
from re import search
from pathlib import Path

from functools import reduce
from io import StringIO
from bs4 import BeautifulSoup

# import pprint
from pprint import pprint 
import re, pprint, random, requests, glob

from scipy.stats import mstats
from operator import itemgetter
from timeit import timeit


# <font color ='orange' > Classification


In [ ]:
#preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, normalize

#Modle Selection
from sklearn.model_selection import train_test_split, cross_val_score

#Evaluation Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score, explained_variance_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support


In [ ]:
# Load Data Set

pd.options.display.max_columns = 250
pd.options.display.max_rows = 2000
pd.options.display.float_format = '{:.2f}'.format

# <font color ='blue' > HighLevel


# <font color ='yellow' > NLP


In [ ]:
#Importing libraries
import nltk, re, pprint, random

from nltk.tokenize import word_tokenize
from sklearn import feature_extraction, linear_model, model_selection, preprocessing


# NLP

In [ ]:

sentences = [
    'i love my dog',
    'I, love my cat',
    'You love my dog!',
    "I am running with dog"
]

tokenizer = Tokenizer(num_words = 1)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

# <font color ='red' > NLP problem 1 : Text


In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])


## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())


train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])


## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
import linear_model
clf = linear_model.RidgeClassifier()

scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores


clf.fit(train_vectors, train_df["target"])

# <font color ='red' > NLP problem 2 : Text


In [ ]:
import spacy
nlp = spacy.load('en')

doc = nlp("Tea is healthy and calming, don't you think?")

for token in doc:
    print(token)
    
print(f"Token \t\tLemma \t\tStopword".format('Token', 'Lemma', 'Stopword'))
print("-"*40)
for token in doc:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}")
    

from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')


